In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
## bUILT-iN pACKAGES
import sys, os, time, bz2, zlib, pickle, math, json, csv
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True)
from IPython.display import display

## lAST sUMMER
from romtoslp import *
from sentences import *
from DCS import *
import MatDB
from heap_n_PrimMST import *

In [10]:
def open_dsbz2(filename):
    with bz2.BZ2File(filename, 'r') as f:
        loader = pickle.load(f)
    
#     conflicts_Dict_correct = loader['conflicts_Dict_correct']
#     nodelist_to_correct_mapping = loader['nodelist_to_correct_mapping']
#     nodelist_correct = loader['nodelist_correct']
#     featVMat_correct = loader['featVMat_correct']
#     featVMat = loader['featVMat']
#     conflicts_Dict = loader['conflicts_Dict']
#     nodelist = loader['nodelist']
    
    return loader

In [125]:
loaded_DCS = pickle.load(open('../Simultaneous_DCS_10K.p', 'rb'))

In [5]:
in_folder = '../NewData/skt_dcs_DS.bz2_10K/'
file_list = os.listdir(in_folder)

In [196]:
f_k = 10
f_name = file_list[f_k]
f_path = os.path.join(in_folder, f_name)
loader = open_dsbz2(f_path)
nodelist = loader['nodelist']
conflicts_Dict = loader['conflicts_Dict']
nodelist_correct = loader['nodelist_correct']
n_nodes = len(nodelist)

dcsObj = loaded_DCS[f_name.replace('.ds.bz2', '.p2')]
print(len(nodelist))

57


In [12]:
loader.keys()

dict_keys(['nodelist', 'nodelist_to_correct_mapping', 'conflicts_Dict', 'featVMat', 'conflicts_Dict_correct', 'nodelist_correct', 'featVMat_correct'])

In [139]:
def mark_further(_partial_sentence, small_nodelist, conflicts_Dict, glob_to_local, full_set, local_i):
    if local_i < len(_partial_sentence):
        while _partial_sentence[local_i] == 2:
            local_i += 1
            if local_i == len(_partial_sentence):
                break
            continue
        
    if local_i == len(_partial_sentence):
#         print(_partial_sentence)
        full_set.append(_partial_sentence)
        return
    
    global_i = small_nodelist[local_i].id
    
    if local_i < len(small_nodelist) - 1:
        partial_sentence = _partial_sentence[:]
        partial_sentence[local_i] = 0
        mark_further(partial_sentence, small_nodelist, conflicts_Dict, glob_to_local, full_set, local_i+1)
    
    partial_sentence = _partial_sentence[:]
    partial_sentence[local_i] = 1
    for conflict in conflicts_Dict[global_i]:
        partial_sentence[glob_to_local[conflict]] = 2
    mark_further(partial_sentence, small_nodelist, conflicts_Dict, glob_to_local, full_set, local_i+1)

In [199]:
max_cid = 0
node_dict = defaultdict(list)
for i in range(len(nodelist)):
    node = nodelist[i]
    node.id = i
    max_cid = max(node.chunk_id, max_cid)
    node_dict[node.chunk_id].append(node)
    

full_words = []
for word in re.split('[ ]+', dcsObj.sentence.strip()):
    full_words.append(rom_slp(word))

valid_set = defaultdict(list)
for cid in range(max_cid + 1):
    word = full_words[cid]
    
    full_set = []
    
    small_nodelist = node_dict[cid]
    glob_to_local = {}
    for i in range(len(small_nodelist)):
        glob_to_local[small_nodelist[i].id] = i
    
    sentence_mark = [0]*len(small_nodelist)
    mark_further(sentence_mark, small_nodelist, conflicts_Dict, glob_to_local, full_set, 0)
    
    
    for S in full_set:
        total_possible_len = 0
        S = np.array(S)
        S[S==2] = 0
        Y = np.where(S)[0]
        nc = Y.shape[0]
        for y in Y:
            total_possible_len += len(small_nodelist[y].derived)
        if total_possible_len >= len(word) - 1:
            repeat = False
            for other_s in valid_set[cid]:
                if np.sum(other_s^S) == len(small_nodelist):
                    repeat = True
            if not repeat:
                valid_set[cid].append(S)
#             print('Valid: ', S)
#         else:
#             print('InValid: ', S)
valid_set = dict(valid_set)
total_valid_sets = 1
for cid, ss in valid_set.items():
    valid_set[cid] = list(map(lambda x: np.where(x)[0], ss))
    print('Options: ', len(valid_set[cid]))
    total_valid_sets *= len(valid_set[cid])
print('Total sentences: ', total_valid_sets)

Options:  18
Options:  1
Options:  87
Options:  348
Total sentences:  544968


In [ ]:
valid_set

In [182]:
nodelist

[WD_Node[C: 0, P: 0, vAc @(3) => vAk],
 WD_Node[C: 0, P: 0, vAc @(30) => vAk],
 WD_Node[C: 0, P: 0, vAgdaRqa @(3) => vAgdaRqa],
 WD_Node[C: 0, P: 8, ja @(31) => jam],
 WD_Node[C: 0, P: 8, ja @(69) => jam],
 WD_Node[C: 0, P: 8, ja @(71) => jam],
 WD_Node[C: 0, P: 3, daRqa @(3) => daRqa],
 WD_Node[C: 1, P: 0, ca @(2) => ca],
 WD_Node[C: 2, P: 0, pAruzya @(31) => pAruzyam],
 WD_Node[C: 2, P: 0, pAruzya @(69) => pAruzyam],
 WD_Node[C: 2, P: 0, pAruzya @(71) => pAruzyam],
 WD_Node[C: 3, P: 0, kroDa @(3) => kroDa],
 WD_Node[C: 3, P: 5, ja @(29) => jas],
 WD_Node[C: 4, P: 0, api @(2) => api],
 WD_Node[C: 5, P: 0, gaRa @(29) => gaRas],
 WD_Node[C: 6, P: 0, azwaka @(29) => azwakas]]

In [136]:
nodelist_correct

['sArAmuKo', 'dIrGaSUko', 'loDraSUkaH', 'suganDikaH']

[WD_Node[C: 0, P: 0, sArAmuKa @(1) => sArAmuKa],
 WD_Node[C: 1, P: 0, dIrGaSUka @(1) => dIrGaSUka],
 WD_Node[C: 2, P: 0, loDraSUka @(1) => loDraSUka],
 WD_Node[C: 3, P: 0, suganDika @(1) => suganDika]]